### Imports

In [1]:
from utils_art import *

from sklearn.datasets import make_classification
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import Isomap
from sklearn.neighbors import KNeighborsTransformer
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression

import pandas as pd
import numpy as np
from textblob import TextBlob
import tempfile
# np.random.seed(0)

from IPython.core.display import Image as image
from PIL import Image
from IPython.display import Image
import plotly.express as px
import plotly
import matplotlib.pyplot as plt
import matplotlib.cm as cm

PLOT_WIDTH=1800
PLOT_HEIGHT=1000
PLOT_AUTOSIZE=True
PCA_TYPE_LIST = ["auto", "full", "arpack", "randomized"]
PLOT_RENDER_LIST = ["png","browser","svg"]
PLOT_RENDER = PLOT_RENDER_LIST[1]
TAG_SELECTION_LIST = ["NNP","NN","JJ"]

import ipywidgets as widgets


IMPORT : utils_art


### Data transformation 

In [2]:
def dfNormalize(df) :
    return ((df - df.mean()) / df.std())

def generateTestSet() :
    X, y = make_classification(
        n_features=24,
        n_classes=5,
        n_samples=1500,
        n_informative=3,
        random_state=5,
        n_clusters_per_class=1,
    )
    return X, y

## Dimention reduction

def dimReduction_TSNE(df, n_components=2, perplexity=30, early_exaggeration =4.0,learning_rate=1000,n_iter=1000,verbose=0,random_state=0,norm_output=False):
    tsne = TSNE(n_components=n_components, perplexity=perplexity,early_exaggeration = early_exaggeration,learning_rate =learning_rate,n_iter=n_iter,verbose=verbose,random_state=random_state)
    out_df = tsne.fit_transform(df)
    out_df = pd.DataFrame(out_df)
    if norm_output :
        out_df = dfNormalize(out_df)
    return out_df

def dimReduction_PCA(df, n_components=2, svd_solver="auto",tol=0.0,whiten=False,random_state=0,norm_output=False):
    pca = PCA(n_components=n_components,svd_solver=svd_solver,tol=tol,random_state=random_state)
    out_df = pca.fit_transform(df)
    out_df = pd.DataFrame(out_df)
    if norm_output :
        out_df = dfNormalize(out_df)
    return out_df

def dimReduction_IPCA(df, n_components=2,whiten=False, batch_size=100,norm_output=False):
    ipca = IncrementalPCA(n_components=n_components,batch_size=batch_size)
    out_df = ipca.fit_transform(df)
    out_df = pd.DataFrame(out_df)
    if norm_output :
        out_df = dfNormalize(out_df)
    return out_df

def dimReduction_NNT(df,n_components=2,n_neighbors=5,mode='distance',algorithm='auto',leaf_size=30,p=2,eigen_solver='auto',tol=0.0,metric='minkowski',n_jobs=None,norm_output=False):
    #KTN mode : 'distance' 'connectivity'
    #KTN algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
    #BOTH n_jobs : None -1
    #BOTH metric = ['minkowski',  'manhattan','cityblock','l1',  'euclidean','l2',  'cosine',  'haversine',  'nan_euclidean'] 'precomputed' ?
    #ISO eigen_solver : ['auto', 'arpack', 'dense']
    cache_path = tempfile.gettempdir()
    knt = KNeighborsTransformer(mode=mode,n_neighbors=n_neighbors,algorithm=algorithm,leaf_size=leaf_size,metric=metric,p=p,n_jobs=n_jobs)
    iso = Isomap(n_components=n_components,n_neighbors=n_neighbors,eigen_solver=eigen_solver,metric=metric,tol=tol,p=p,n_jobs=n_jobs)# 
    nnt = make_pipeline(knt,iso,memory=cache_path)
    out_df = nnt.fit_transform(df)
    out_df = pd.DataFrame(out_df)
    if norm_output :
        out_df = dfNormalize(out_df)
    return out_df

def generateDirRed(mat_emb,df_main,n_components=2,norm_output=False,active_sel=[True,False,False,False]) :
    out_list = []
    out_list_label = []
    if active_sel[0] :
        df_tsne = dimReduction_TSNE(mat_emb,n_components,norm_output=norm_output)
        df_tsne_j = df_main.join(df_tsne, how="inner")
        out_list.append(df_tsne_j)
        out_list_label.append("TSNE")
    if active_sel[1] :
        df_pca = dimReduction_PCA(mat_emb,n_components,norm_output=norm_output)
        df_pca_j = df_main.join(df_pca, how="inner")
        out_list.append(df_pca_j)
        out_list_label.append("PCA")
    if active_sel[2] :
        df_ipca = dimReduction_IPCA(mat_emb,n_components,norm_output=norm_output)
        df_ipca_j = df_main.join(df_ipca, how="inner")
        out_list.append(df_ipca_j)
        out_list_label.append("IPCA")
    if active_sel[3] :
        df_nnt = dimReduction_NNT(mat_emb,n_components,norm_output=norm_output)
        df_nnt_j = df_main.join(df_nnt, how="inner")
        out_list.append(df_nnt_j)
        out_list_label.append("NNT")

    return out_list, out_list_label


### Visualization Functions

In [3]:
## Visualization using "plotly"

def renderAllOptions(df,BD=True,confList=[],label="") :
    plt_list = []
    for conf in confList :
        main_conf = createdefconfdict()|conf
        if label != "" :
            main_conf['title'] = main_conf['title'] + "  -  " + label
        if (BD) :
            plt = plot2D(df,main_conf)
        else :
            plt = plot3D(df,main_conf)
        plt_list.append(plt)
    return plt_list

# 2D plots : 

def plot2D(df,conf_dict={}) : # Scatter
    fig = px.scatter(df, x=conf_dict["x"], y=conf_dict["y"], color=conf_dict["c"],size=conf_dict["size"],
        symbol=conf_dict["symbol"],text=conf_dict["text"], 
        hover_name=conf_dict["h_name"], hover_data=conf_dict["h_data"], custom_data=conf_dict["c_data"], 
        facet_row=conf_dict["facet_r"], facet_col=conf_dict["facet_c"],facet_col_wrap=conf_dict["facet_cw"],
        facet_row_spacing=conf_dict["facet_rs"], facet_col_spacing=conf_dict["facet_cs"],
        animation_frame=conf_dict["animation_frame"], animation_group=conf_dict["animation_group"],
        marginal_x=conf_dict["marginal_x"], marginal_y=conf_dict["marginal_y"],
        trendline=conf_dict["trendline"], log_x=conf_dict["log_x"],log_y=conf_dict["log_y"], render_mode=conf_dict["render_mode"],
        size_max=conf_dict["size_max"], opacity=conf_dict["opacity"]) 
    # fig.update_annotations("bgcolor":rgba(13,57,F2,0),captureevents:test_val)
    fig.update_layout(title=conf_dict["title"], xaxis_title=conf_dict["xtitle"], yaxis_title=conf_dict["ytitle"],width=PLOT_WIDTH,height=PLOT_HEIGHT,autosize=PLOT_AUTOSIZE)
    
    if type(conf_dict["browser"]) == type(None) :
        fig.show()
    else :
        fig.show(renderer="browser")
        widgets.IntSlider()
    return fig

# def linegraph(df) :
#     # df = px.data.gapminder().query("country=='Canada'")
#     fig = px.line(df, x='year', y='words', color='category')#_month , symbol="url_TLD"_mont
#     fig.show()
# #     fig = px.line(df, x="year_month", title='TEST',color='category')
# #     fig.show()
def plot3D(df,conf_dict={}) :
    fig = px.scatter_3d(df, x=conf_dict["x"], y=conf_dict["y"], z=conf_dict["z"], color=conf_dict["c"],size=conf_dict["size"],
        symbol=conf_dict["symbol"],text=conf_dict["text"],
        hover_name=conf_dict["h_name"], hover_data=conf_dict["h_data"],custom_data=conf_dict["c_data"],
        animation_frame=conf_dict["animation_frame"], animation_group=conf_dict["animation_group"],
        log_x=conf_dict["log_x"],log_y=conf_dict["log_y"],log_z=conf_dict["log_z"],
        size_max=conf_dict["size_max"], opacity=conf_dict["opacity"])
    fig.update_layout(title=conf_dict["title"], xaxis_title=conf_dict["xtitle"], yaxis_title=conf_dict["ytitle"],width=PLOT_WIDTH,height=PLOT_HEIGHT,autosize=PLOT_AUTOSIZE) #, zaxis_title=conf_dict["ztitle"]
    if type(conf_dict["browser"]) == type(None) :
        fig.show()
    else :
        fig.show(renderer="browser")
    return fig

# ["x","y","z","c","size","symbol","text","h_name","h_data","c_data","title","xtitle","ytitle"]
def createdefconfdict() :
    ret_dict = {}
    val_list = ["x","y","z","c","size","symbol","h_name","h_data","c_data","text","facet_r","facet_c","facet_cw","facet_rs","facet_cs","title","xtitle","ytitle","ztitle","browser","animation_frame","animation_group","marginal_x","marginal_y","trendline","log_x","log_y","log_z","render_mode","size_max","opacity"]
    for val in val_list :
        ret_dict[val] = None
    return ret_dict

## Visualization using "pandas"

def plot2Dpandas(df,size=3,color="#a98d19",width=15,height=15,title="default",xl="x",yl="y",save=False,path="",savecount=0) :
    res = df.plot.scatter(x = 0, y = 1, s=size, c="o_token_input",figsize=(width,height),title=title,xlabel=xl,ylabel=yl).get_figure(); 
    if save :
        res.savefig(path+"fig_"+str(savecount)+".png")
        
## Visualization using "matplotlib"

def plot2Dmatplotlib(df) :
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.scatter(df[0], df[1], 0.5,"#a98d19")#, c=df['color']
    plt.show()
    
    
##
def testDimReduct(df,i) :
    # df = dfNormalize(df)
    df_tsne = dimReduction_TSNE(df)
    plot2Dpandas(df_tsne,title="TSNE (T-distributed Stochastic Neighbor Embedding)",save=True,path=folder_path_graph+"TSNE_",savecount=i)
    
    df_pca = dimReduction_PCA(df)
    plot2Dpandas(df_pca,title="PCA (Principal Component Analysis)",save=True,path=folder_path_graph+"PCA_",savecount=i)
    
    df_ipca = dimReduction_IPCA(df)
    plot2Dpandas(df_ipca,title="IPCA (Incremental Principal Component Analysis)",save=True,path=folder_path_graph+"IPCA_",savecount=i)
    
    df_nnt = dimReduction_NNT(df)
    plot2Dpandas(df_nnt,title="NNT (Nearest Neighbors Transformer)",save=True,path=folder_path_graph+"IPCA_",savecount=i)

def addVisuColumns(df) :
    if "url_TLD" in df.columns :
        df["bool_url"] = np.where(df['url_TLD'] == "com", True, False)
        #df["bool_url"] = df["url_TLD"] == "com"
    df["bool_tb_pos"] = np.where(df['tb.pos'] > df['tb.neg'], "y", "n")
    df["bool_vs_pos"] = np.where(df['vs.pos'] > df['vs.neg'], "y", "n")
    df["bool_ts_pos"] = np.where(df['ts.pos'] > df['ts.neg'], "y", "n")
    df["bool_vs_neu"] = np.where(df['vs.neu'] > 0, "y", "n")
    df["bool_vs_comp"] = np.where(df['vs.comp'] > 0, "y", "n")
    df['bool_sent_all'] = df[["bool_tb_pos","bool_vs_pos","bool_ts_pos","bool_vs_neu","bool_vs_comp"]].agg(''.join, axis=1)
    return df

def update_point(trace, points, selector):
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    for i in points.point_inds:
        c[i] = '#bae2be'
        s[i] = 20
        with f.batch_update():
            scatter.marker.color = c
            scatter.marker.size = s

### Variables

In [ ]:
# df_main = openDFcsv(folder_path_input_df,filename_input_df)
# df_main2 = addVisuColumns(df_main)
# display(df_main)
# saveDFcsv(df_main2, folder_path_input_df, filename_input_df+"_viz_col1",True)

In [4]:
folder_path_input_df="C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/1_4_join_main/test/"
folder_path_input_embd="C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/2_1_embdedding_main/test/"
filename_input_df="join1_file"
filename_input_embd="embdedding_file"

folder_path_graph="C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/3_2_visu_main/"


In [5]:
# x = None
# y = None
# z = None
marginal_list = [None,'rug', 'box', 'violin','histogram'] #'box', 'violin','histogram'
trendline_list = [None,'ols', 'lowess', 'rolling', 'expanding','ewm']
render_mode_list =['auto', 'svg','webgl']
category = "category"#""bool_sent_all"#"category"
size = "words"#"text_len"  #"text_len"#"words"  #"text_len"#
symbol = None#  "" #None #  "url_TLD"
h_name = "source_title"  #"title_quer" #None #
h_data = []#"title_quer","subjectivity","polarity","pos1","neg1","0_s_k","0_s_t"] # None# "title_quer"#None #"source_title"
c_data = None  #"word_combined_s_k"
browser = True 
facet_r = None #"year_month" #"category"
facet_c = None
facet_rs =0.03
facet_cs =0.03
animation_frame = "year"#"year_month"
animation_group = None#"hash_key"
marginal_x = marginal_list[0]
marginal_y = marginal_list[0]
trendline = trendline_list[0]
log_x =False
log_y = False
log_z = False
render_mode = render_mode_list[0]
size_max = 30
opacity=0.9

_2d_embd = {"x":0,"y":1,'c':category,'size':size,'symbol':symbol, "h_name":h_name,"h_data":h_data,"c_data":c_data,"text":None, 'title':"2d_embd",
        "facet_r":facet_r,"facet_c":facet_c,"facet_cw":None,'facet_rs':facet_rs,'facet_cs':facet_cs,"browser":browser,
           "animation_frame":animation_frame,"animation_group":animation_group,"marginal_x":marginal_x,"marginal_y":marginal_y,
           "trendline":trendline,"log_x":log_x,"log_y":log_y,"render_mode":render_mode,"size_max":size_max,"opacity":opacity,
           'title':"2d_embd","xtitle":None,"ytitle":None}
          
_3d_embd = {"x":0,"y":1,"z":2,'c':category,'size':size,'symbol':symbol,
            "h_name":h_name,"h_data":h_data,"c_data":c_data,"text":None,"browser":browser,
            "animation_frame":animation_frame,"animation_group":animation_group,"size_max":size_max,"opacity":opacity,
            "log_x":log_x,"log_y":log_y,"log_z":log_z,
            'title':"3d_embd","xtitle":None,"ytitle":None}

_2d_embd_basic = {"x":0,"y":1,'c':"category","title":"Scatter plot of dimention reduced embeding data from articles","xtitle":"Component 1","ytitle":"Component 2","browser":browser,'size':size,"size_max":size_max}
_3d_embd_basic = {"x":0,"y":1, 'z':2,'c':"category","title":"Scatter plot of dimention reduced embeding data from articles","xtitle":"Component 1","ytitle":"Component 2","ztitle":"Component 3","browser":browser,'size':size,"size_max":size_max}
_2d_sent_basic = {"x":"tb.polaj","y":"tb.sub",'c':"category","title":"Scatter plot of the 'polarity' and 'objectivity' of article data","xtitle":"Polarity (0-1)","ytitle":"Subjectivity (0-1)","browser":browser,'size':size,"size_max":size_max,"marginal_x":marginal_list[3],"marginal_y":marginal_list[4]}
_3d_sent_basic = {"x":"tb.polaj","y":"tb.sub","z":"ts.pos",'c':"category","title":"Scatter plot of the 'polarity', 'objectivity' and 'positivity' of article data","xtitle":"Polarity (0-1)","ytitle":"Subjectivity (0-1)","ztitle":"Positivity (0-1)","browser":browser,'size':size,"size_max":size_max}
_2d_len_basic = {"x":"sentences","y":"noun_phrases",'c':"category","title":"Scatter plot of text nature and volume data from articles","xtitle":"Number of Sentences","ytitle":"Number of Nouns","browser":browser,'size':size,"size_max":size_max}
_3d_len_basic = {"x":"sentences","y":"noun_phrases", 'z':"words",'c':"category","title":"Scatter plot of text nature and volume data from articles","xtitle":"Number of Sentences","ytitle":"Number of Nouns","ztitle":"Number of Words","browser":browser,'size':size,"size_max":size_max}
_2dList_basic = [_2d_embd_basic ,_2d_sent_basic ,_2d_len_basic]
_3dList_basic = [_3d_embd_basic ,_3d_sent_basic, _3d_len_basic]

_2d_embd_test = {"x":0,"y":1,'c':"bool_sent_all","title":"Scatter plot of dimention reduced embeding data from articles","xtitle":"Component 1","ytitle":"Component 2","browser":browser,'size':size,"size_max":size_max}
_2dList_test = [_2d_embd]+_2dList_basic
_3dList_test = [_3d_embd]+_3dList_basic

# Main

In [6]:
# Load both embeded matrix and stat dataframe
df_main = openDFcsv(folder_path_input_df,filename_input_df)
mat_emb = openDFcsv(folder_path_input_embd,filename_input_embd)
mat_emb.drop(["hash_key"], axis=1,inplace=True)
# linegraph(df_main)
out_list, out_list_label = generateDirRed(mat_emb, df_main,3, norm_output=False, active_sel=[True,True,False,False])
print(len(out_list))
for i in range(len(out_list)):
    plt_2dList = renderAllOptions(out_list[i],True,_2dList_test,out_list_label[i])
    plt_3dList = renderAllOptions(out_list[i],False,_3dList_test,out_list_label[i])
print(plt_2dList)
# scatter.on_click(update_point)    


ValueError: could not convert string to float: 'chatcmpl-97WM90efNtrZ4f4KhJNZ3gHkIsaQD'

In [ ]:
df_main = openDFcsv(folder_path_input_df,filename_input_df)
mat_emb = openDFcsv(folder_path_input_embd,filename_input_embd)
mat_emb.drop(["hash_key"], axis=1,inplace=True)
df_tsne = dimReduction_TSNE(mat_emb)
print(df_tsne)

In [ ]:
#Create anotation for Link
plot = plt_2dList[0]
plot.add_annotation(
    x=2.2, y=40,
    xref="x", yref="y",
    text="<a href='https://python-graph-gallery.com'>Click here!</a>",
    showarrow=False,
    font=dict(size=14, color="blue"),
    align="center",
    bgcolor="rgba(255, 255, 0, 0.5)",
)

In [ ]:
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


app = Dash(__name__)
df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')
app.layout = html.Div([
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[0])
])

if __name__ == '__main__':
    app.run_server(port=2223,debug=True)

In [ ]:
# AGG Graphs
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import numpy as np
import dash
import webbrowser
from threading import Timer

app = dash.Dash(__name__) 

app.layout = html.Div([
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[0]),
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[1]),
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[2]),
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[3]),
])


port = 5000 # or simply open on the default `8050` port

def open_browser():
    webbrowser.open_new("http://localhost:{}".format(port))

if __name__ == '__main__':
    Timer(1, open_browser).start();
    app.run_server(debug=True, port=port)

In [ ]:
# Markdown
from dash import html
app = Dash(__name__)
html.Div([
    html.H1('Hello Dash'),
    html.Div([
        html.P('Dash converts Python classes into HTML'),
        html.P("This conversion happens behind the scenes by Dash's JavaScript front-end")
    ])
])
if __name__ == '__main__':
    app.run_server(port=2223,debug=True)

##### 

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import numpy as np

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Interactive plot with custom data source'),
    dcc.Graph(id="graph"),
    html.P("Number of bars:"),
    dcc.Slider(id="slider", min=2, max=10, value=4, step=1),
])


@app.callback(
    Output("graph", "figure"), 
    Input("slider", "value"))
def update_bar_chart(size):
    data = np.random.normal(3, 2, size=size) # replace with your own data source
    fig = go.Figure(
        data=[go.Bar(y=data)],
        layout_title_text="Native Plotly rendering in Dash"
    )
    return fig

app.run_server(port=2224, debug=True)

In [ ]:
from dash import Dash, html, dcc

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),
    dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=plt_2dList[0]
    )
])

if __name__ == '__main__':
    app.run(port=2226,debug=True)

In [ ]:
plot_url.show()

### Keyword functions

In [ ]:
from dash import Dash, html, dcc

app = Dash(__name__)

app.layout = html.Div([
    html.Div(children=[
        html.Label('Dropdown'),
        dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),

        html.Br(),
        html.Label('Multi-Select Dropdown'),
        dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'],
                     ['Montréal', 'San Francisco'],
                     multi=True),

        html.Br(),
        html.Label('Radio Items'),
        dcc.RadioItems(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),
    ], style={'padding': 10, 'flex': 1}),

    html.Div(children=[
        html.Label('Checkboxes'),
        dcc.Checklist(['New York City', 'Montréal', 'San Francisco'],
                      ['Montréal', 'San Francisco']
        ),

        html.Br(),
        html.Label('Text Input'),
        dcc.Input(value='MTL', type='text'),

        html.Br(),
        html.Label('Slider'),
        dcc.Slider(
            min=0,
            max=9,
            marks={i: f'Label {i}' if i == 1 else str(i) for i in range(1, 6)},
            value=5,
        ),
    ], style={'padding': 10, 'flex': 1})
], style={'display': 'flex', 'flexDirection': 'row'})

if __name__ == '__main__':
    app.run(port=2229,debug=True)

### Keyword Workflow

In [ ]:
from dash import Dash, dcc, html

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=plt_2dList[0]
    )
])

if __name__ == '__main__':
    app.run(port=2227,debug=True)

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import numpy as np
import dash
import webbrowser
from threading import Timer

app = dash.Dash(__name__) 

app.layout = html.Div([
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[0]),
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[1]),
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[2]),
    dcc.Graph(id='life-exp-vs-gdp',figure=plt_2dList[3]),
])


port = 5000 # or simply open on the default `8050` port

def open_browser():
    webbrowser.open_new("http://localhost:{}".format(port))

if __name__ == '__main__':
    Timer(1, open_browser).start();
    app.run_server(debug=True, port=port)

In [ ]:
import plotly.graph_objects as go

import pandas as pd

# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
import plotly.graph_objects as go

import pandas as pd

# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

fig = go.Figure(data=[go.Surface(z=z_data.values)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


# This dataframe has 244 lines, but 4 distinct values for `day`
df = px.data.tips()
pie = px.pie(df, values='tip', names='day')
fig=make_subplots(rows=1, cols=2,
                  specs=[[{"type": "domain"},{"type": "xy"}]])

fig.add_trace(pie.data[0], row=1, col=1)
fig.add_trace(go.Bar(x=['A', 'B', 'C'], y=[25, 17, 19], marker_color="blue", name= "Quantity"), row=1, col=2)
fig.update_layout(width=700, height=350, bargap=0.05)

fig.add_annotation(dict(x=0.22, y=-0.1,   ax=0, ay=0,
                    xref = "paper", yref = "paper", 
                    text= "Hello, pie chart!"
                  ))
fig.add_annotation(dict(x=0.72, y=-0.14,   ax=0, ay=0,
                    xref = "paper", yref = "paper", 
                    text= "My Bars!"
                  ))

## Test

### Display Test

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *

trace1 = Scatter3d(
    x=[1, 2],
    y=[1, 2],
    z=[1, 2],
    name='Legendary'
)
data = Data([trace1])
layout = Layout(
    showlegend=True,
    scene=Scene(
        xaxis=XAxis(title='x axis title'),
        yaxis=YAxis(title='y axis title'),
        zaxis=ZAxis(title='z axis title')
    )
)

In [ ]:
df_open = openDFcsv(folder_path_embd, filename_embd)
df_open.set_index("hash_key", inplace=True) # index

# np_data = modelDf(df_open,True)
for perplexity in [5,20,50] :
    for early_exaggeration in [4.0] :#[2.0,4.0,6.0] :
        for learning_rate in [700] : # [200,600,1000] :
            for n_iter in [400,1000,2500] :
                print("perplexity",perplexity)
                print("early_exaggeration",early_exaggeration)
                print("learning_rate",learning_rate)
                print("n_iter",n_iter)
                np_data = TSNE(n_components=2,perplexity = perplexity,early_exaggeration=early_exaggeration,learning_rate=learning_rate,n_iter=n_iter, random_state=0).fit_transform(df_open)
                plot2Dminimal(np_data)

In [ ]:

# _3d_pos = {"x":"tb.polaj","y":"tb.sub","z":"ts.pos",'c':category,'size':size,'symbol':symbol,
#             "h_name":h_name,"h_data":h_data,"c_data":c_data,"text":None,"browser":browser,
#             "animation_frame":animation_frame,"animation_group":animation_group,"size_max":size_max,"opacity":opacity,
#             "log_x":log_x,"log_y":log_y,"log_z":log_z,
#             'title':"3d_embd","xtitle":None,"ytitle":None}

#{"x":"tb.polaj","y":"tb.sub",'c':category,'size':size,'symbol':symbol,
#{"x":"tb.polaj","y":"tb.sub","z":"ts.pos",'c':category,'size':size,'symbol':symbol,

# _3d_embd_nlp = {"x":"tb.sub","y":"tb.polaj",'z':"ts.pos",'c':"category",'size':size,'title':"3d_embd","h_data":h_data,"size_max":size_max,"opacity":opacity,"browser":browser}
# _3d_nlp = {"x":"polarity","y":"subjectivity",'z':"compound",'c':"category",'size':size,'symbol':symbol,'title':"3d_nlp"}
# _3d_nlp_sent = {"x":"pos1","y":"neu1",'z':"neg1",'c':"category",'size':size,'symbol':symbol,'title':"3d_nlp_sent"}
# _3d_len = {"x":"words","y":"noun_phrases",'z':"sentences",'c':"category",'size':size,'symbol':symbol,'title':"3d_len"}


In [ ]:
# folder_path="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/"
# # folder_path="C:/Users/User/OneDrive/Desktop/article/file_2/test_llm_output/run2/"
# folder_path_embd="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/embd/only_embd/"
# folder_path_keyword="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/keyword/"
# folder_path_llm="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/llm/"
# folder_path_graph="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/graphs/"
# folder_path_viz="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/viz/"

# filename="article_stats_embedding"
# filename_embd="embedding_matrix_1000"#"embedding_matrix_main"#"article_stats_embedding_main" #""
# filename_keyw="article_keyword_main"
# open_path_keyw="C:/Users/User/OneDrive/Desktop/article/file_2/join_1_df/"

In [ ]:
import os
from threading import Timer
import webbrowser

import dash
from dash import html
from dash import dcc

app = dash.Dash(__name__)

app.layout = html.Div(
    [
        dcc.DatePickerRange(id='date-range')
    ]
)

def open_browser():
    if not os.environ.get("WERKZEUG_RUN_MAIN"):
        webbrowser.open_new('http://127.0.0.1:1222/')

if __name__ == "__main__":
    Timer(1, open_browser).start()
    app.run_server(debug=True, port=1222)